In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint
import sys
import itertools

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/.local/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
TRAINED_MODEL_DIR="best_trained_model"
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [4]:
TRAINED_MODEL_DIR

'best_trained_model'

In [5]:
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [6]:
from finrl.meta.env_stock_trading.env_stocktrading import *

In [7]:
from __future__ import annotations
import random

In [8]:
class StockTradingEnv2(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {"render.modes": ["human"]}
    def __init__(
        self,
        df: pd.DataFrame,
        stock_dim: int,
        hmax: int,
        initial_amount: int,
        num_stock_shares: list[int],
        buy_cost_pct: list[float],
        sell_cost_pct: list[float],
        reward_scaling: float,
        state_space: int,
        action_space: int,
        tech_indicator_list: list[str],
        turbulence_threshold=None,
        risk_indicator_col="turbulence",
        make_plots: bool = False,
        print_verbosity=10,
        day=0,
        initial=True,
        previous_state=[],
        model_name="",
        mode="",
        iteration="",
        random_day=None,
        reset_interval=None,
    ):
        self.day = day
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.num_stock_shares = num_stock_shares
        self.initial_amount = initial_amount  # get the initial cash
        self.buy_cost_pct = buy_cost_pct
        self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.state_space,))
        self.data = self.df.loc[self.day, :]
        self.terminal = False
        self.make_plots = make_plots
        self.print_verbosity = print_verbosity
        self.turbulence_threshold = turbulence_threshold
        self.risk_indicator_col = risk_indicator_col
        self.initial = initial
        self.previous_state = previous_state
        self.model_name = model_name
        self.mode = mode
        self.iteration = iteration
        self.random_day=random_day
        self.reset_day=reset_interval
        self.reset_interval=reset_interval
        # initalize state
        self.state = self._initiate_state()

        # initialize reward
        self.reward = 0
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.episode = 0
        # memorize all the total balance change
        self.asset_memory = [
            self.initial_amount
            + np.sum(
                np.array(self.num_stock_shares)
                * np.array(self.state[1 : 1 + self.stock_dim])
            )
        ]  # the initial total asset is calculated by cash + sum (num_share_stock_i * price_stock_i)
        self.rewards_memory = []
        self.actions_memory = []
        self.state_memory = (
            []
        )  # we need sometimes to preserve the state in the middle of trading process
        self.date_memory = [self._get_date()]
        #         self.logger = Logger('results',[CSVOutputFormat])
        # self.reset()
        self._seed()
        print(f"Intialize Environment StartDay: {self.day}, ResetDay: {self.reset_day},Episode: {self.episode}")

    def _sell_stock(self, index, action):
        def _do_sell_normal():
            if (self.state[index + 2 * self.stock_dim + 1] != True):
                # check if the stock is able to sell, for simlicity we just add it in techical index
                # if self.state[index + 1] > 0: # if we use price<0 to denote a stock is unable to trade in that day, the total asset calculation may be wrong for the price is unreasonable
                # Sell only if the price is > 0 (no missing data in this particular date)
                # perform sell action based on the sign of the action
                if self.state[index + self.stock_dim + 1] > 0:
                    # Sell only if current asset is > 0
                    sell_num_shares = min(abs(action), self.state[index + self.stock_dim + 1])
                    sell_amount = (self.state[index + 1]* sell_num_shares* (1 - self.sell_cost_pct[index]))
                    # update balance
                    self.state[0] += sell_amount

                    self.state[index + self.stock_dim + 1] -= sell_num_shares
                    self.cost += (self.state[index + 1]* sell_num_shares* self.sell_cost_pct[index])
                    self.trades += 1
                else:
                    sell_num_shares = 0
            else:
                sell_num_shares = 0

            return sell_num_shares

        # perform sell action based on the sign of the action
        if self.turbulence_threshold is not None:
            if self.turbulence >= self.turbulence_threshold:
                if self.state[index + 1] > 0:
                    # Sell only if the price is > 0 (no missing data in this particular date)
                    # if turbulence goes over threshold, just clear out all positions
                    if self.state[index + self.stock_dim + 1] > 0:
                        # Sell only if current asset is > 0
                        sell_num_shares = self.state[index + self.stock_dim + 1]
                        sell_amount = (self.state[index + 1]* sell_num_shares* (1 - self.sell_cost_pct[index]))
                        # update balance
                        self.state[0] += sell_amount
                        self.state[index + self.stock_dim + 1] = 0
                        self.cost += (
                            self.state[index + 1]
                            * sell_num_shares
                            * self.sell_cost_pct[index]
                        )
                        self.trades += 1
                    else:
                        sell_num_shares = 0
                else:
                    sell_num_shares = 0
            else:
                sell_num_shares = _do_sell_normal()
        else:
            sell_num_shares = _do_sell_normal()

        return sell_num_shares

    def _buy_stock(self, index, action):
        def _do_buy():
            if (self.state[index + 2 * self.stock_dim + 1] != True):  # check if the stock is able to buy
                # if self.state[index + 1] >0:
                # Buy only if the price is > 0 (no missing data in this particular date)
                available_amount = self.state[0] // (
                    self.state[index + 1] * (1 + self.buy_cost_pct[index])
                )  # when buying stocks, we should consider the cost of trading when calculating available_amount, or we may be have cash<0
                # print('available_amount:{}'.format(available_amount))

                # update balance
                buy_num_shares = min(available_amount, action)
                buy_amount = (self.state[index + 1]* buy_num_shares* (1 + self.buy_cost_pct[index]))
                self.state[0] -= buy_amount

                self.state[index + self.stock_dim + 1] += buy_num_shares

                self.cost += (self.state[index + 1] * buy_num_shares * self.buy_cost_pct[index])
                self.trades += 1
            else:
                buy_num_shares = 0

            return buy_num_shares

        # perform buy action based on the sign of the action
        if self.turbulence_threshold is None:
            buy_num_shares = _do_buy()
        else:
            if self.turbulence < self.turbulence_threshold:
                buy_num_shares = _do_buy()
            else:
                buy_num_shares = 0
                pass

        return buy_num_shares

    def _make_plot(self):
        plt.plot(self.asset_memory, "r")
        plt.savefig(f"results/account_value_trade_{self.episode}.png")
        plt.close()

    def step(self, actions):
        
        if self.reset_day:
            self.terminal = self.day >= self.reset_day
            if self.terminal:
                print(f"Environment reached Terminal state as number of trading days reached limit!!")
        else:
            self.terminal = self.day >= len(self.df.index.unique()) - 1
            print(f"Environment reached Terminal state as number of trading days reached data limit!!")
            
        if self.terminal:
            # print(f"Episode: {self.episode}")
            if self.make_plots:
                self._make_plot()
            end_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            df_total_value = pd.DataFrame(self.asset_memory)
            tot_reward = ( self.state[0] + sum(
                    np.array(self.state[1 : (self.stock_dim + 1)])
                    * np.array(
                        self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]
                    )
                )
                - self.asset_memory[0]
            )  # initial_amount is only cash part of our initial asset
            df_total_value.columns = ["account_value"]
            df_total_value["date"] = self.date_memory
            df_total_value["daily_return"] = df_total_value["account_value"].pct_change(1)
            if df_total_value["daily_return"].std() != 0:
                sharpe = ((252**0.5)* df_total_value["daily_return"].mean()/ df_total_value["daily_return"].std())
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.columns = ["account_rewards"]
            df_rewards["date"] = self.date_memory[:-1]
            if self.episode % self.print_verbosity == 0:
                print(f"day: {self.day}, episode: {self.episode}")
                print(f"begin_total_asset: {self.asset_memory[0]:0.2f}")
                print(f"end_total_asset: {end_total_asset:0.2f}")
                print(f"total_reward: {tot_reward:0.2f}")
                print(f"total_cost: {self.cost:0.2f}")
                print(f"total_trades: {self.trades}")
                if df_total_value["daily_return"].std() != 0:
                    print(f"Sharpe: {sharpe:0.3f}")
                print("=================================")

            if (self.model_name != "") and (self.mode != ""):
                df_actions = self.save_action_memory()
                df_actions.to_csv(
                    "results/actions_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    )
                )
                df_total_value.to_csv(
                    "results/account_value_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                df_rewards.to_csv(
                    "results/account_rewards_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                plt.plot(self.asset_memory, "r")
                plt.savefig(
                    "results/account_value_{}_{}_{}.png".format(
                        self.mode, self.model_name, self.iteration
                    )
                )
                plt.close()

            return self.state, self.reward, self.terminal, {}

        else:
            actions = actions * self.hmax  # actions initially is scaled between 0 to 1
            actions = actions.astype(
                int
            )  # convert into integer because we can't by fraction of shares
            if self.turbulence_threshold is not None:
                if self.turbulence >= self.turbulence_threshold:
                    actions = np.array([-self.hmax] * self.stock_dim)
            begin_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            # print("begin_total_asset:{}".format(begin_total_asset))

            argsort_actions = np.argsort(actions)
            sell_index = argsort_actions[: np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][: np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print(f"Num shares before: {self.state[index+self.stock_dim+1]}")
                # print(f'take sell action before : {actions[index]}')
                actions[index] = self._sell_stock(index, actions[index]) * (-1)
                # print(f'take sell action after : {actions[index]}')
                # print(f"Num shares after: {self.state[index+self.stock_dim+1]}")

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                actions[index] = self._buy_stock(index, actions[index])

            self.actions_memory.append(actions)

            # state: s -> s+1
            #print(f"Trading Day {self.day}")
            self.day += 1
            self.data = self.df.loc[self.day, :]
            if self.turbulence_threshold is not None:
                if len(self.df.tic.unique()) == 1:
                    self.turbulence = self.data[self.risk_indicator_col]
                elif len(self.df.tic.unique()) > 1:
                    self.turbulence = self.data[self.risk_indicator_col].values[0]
            self.state = self._update_state()

            end_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            self.asset_memory.append(end_total_asset)
            self.date_memory.append(self._get_date())
            self.reward = end_total_asset - begin_total_asset
            self.rewards_memory.append(self.reward)
            self.reward = self.reward * self.reward_scaling
            self.state_memory.append(self.state)  # add current state in state_recorder for each step

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        # initiate state
        self.state = self._initiate_state()

        if self.initial:
            self.asset_memory = [
                self.initial_amount
                + np.sum(
                    np.array(self.num_stock_shares)
                    * np.array(self.state[1 : 1 + self.stock_dim])
                )
            ]
        else:
            previous_total_asset = self.previous_state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(
                    self.previous_state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]
                )
            )
            self.asset_memory = [previous_total_asset]
            
        ## Choose a random day for start of trading    
        if self.random_day:
            self.day = random.randint(0,len(self.df.index.unique())-self.reset_interval-1)
            if self.reset_day:
                self.reset_day=self.day+self.reset_interval
        else:
            self.day = 0
            
        self.data = self.df.loc[self.day, :]
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.terminal = False
        # self.iteration=self.iteration
        self.rewards_memory = []
        self.actions_memory = []
        self.date_memory = [self._get_date()]

        self.episode += 1
        print(f"Reseting Environment StartDay: {self.day}, ResetDay: {self.reset_day},Episode: {self.episode}")

        return self.state

    def render(self, mode="human", close=False):
        return self.state

    def _initiate_state(self):
        if self.initial:
            # For Initial State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.initial_amount]
                    + self.data.close.values.tolist()
                    + self.num_stock_shares
                    + sum(
                        (
                            self.data[tech].values.tolist()
                            for tech in self.tech_indicator_list
                        ),
                        [],
                    )
                )  # append initial stocks_share to initial state, instead of all zero
            else:
                # for single stock
                state = (
                    [self.initial_amount]
                    + [self.data.close]
                    + [0] * self.stock_dim
                    + sum(([self.data[tech]] for tech in self.tech_indicator_list), [])
                )
        else:
            # Using Previous State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.previous_state[0]]
                    + self.data.close.values.tolist()
                    + self.previous_state[
                        (self.stock_dim + 1) : (self.stock_dim * 2 + 1)
                    ]
                    + sum(
                        (
                            self.data[tech].values.tolist()
                            for tech in self.tech_indicator_list
                        ),
                        [],
                    )
                )
            else:
                # for single stock
                state = (
                    [self.previous_state[0]]
                    + [self.data.close]
                    + self.previous_state[
                        (self.stock_dim + 1) : (self.stock_dim * 2 + 1)
                    ]
                    + sum(([self.data[tech]] for tech in self.tech_indicator_list), [])
                )
        return state

    def _update_state(self):
        if len(self.df.tic.unique()) > 1:
            # for multiple stock
            state = (
                [self.state[0]]
                + self.data.close.values.tolist()
                + list(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
                + sum(
                    (
                        self.data[tech].values.tolist()
                        for tech in self.tech_indicator_list
                    ),
                    [],
                )
            )

        else:
            # for single stock
            state = (
                [self.state[0]]
                + [self.data.close]
                + list(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
                + sum(([self.data[tech]] for tech in self.tech_indicator_list), [])
            )

        return state

    def _get_date(self):
        if len(self.df.tic.unique()) > 1:
            date = self.data.date.unique()[0]
        else:
            date = self.data.date
        return date

    # add save_state_memory to preserve state in the trading process
    def save_state_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            state_list = self.state_memory
            df_states = pd.DataFrame(
                state_list,
                columns=[
                    "cash",
                    "Bitcoin_price",
                    "Gold_price",
                    "Bitcoin_num",
                    "Gold_num",
                    "Bitcoin_Disable",
                    "Gold_Disable",
                ],
            )
            df_states.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            state_list = self.state_memory
            df_states = pd.DataFrame({"date": date_list, "states": state_list})
        # print(df_states)
        return df_states

    def save_asset_memory(self):
        date_list = self.date_memory
        asset_list = self.asset_memory
        df_account_value = pd.DataFrame(
            {"date": date_list, "account_value": asset_list}
        )
        return df_account_value

    def save_action_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            action_list = self.actions_memory
            df_actions = pd.DataFrame(action_list)
            df_actions.columns = self.data.tic.values
            df_actions.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            action_list = self.actions_memory
            df_actions = pd.DataFrame({"date": date_list, "actions": action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs


In [9]:
from datetime import datetime, timedelta
def get_nth_previous_date(n):
    today = datetime.today()
    nth_previous_date = today - timedelta(days=n)
    return nth_previous_date.strftime('%Y-%m-%d')


In [10]:
#validate on last 10 days
n=10
TRAIN_END_DATE=TEST_START_DATE=get_nth_previous_date(n)
TRAIN_END_DATE

'2023-02-11'

In [11]:
TRAIN_START_DATE = '2005-04-01'
# Test date is something unreachable in this lifetime
TEST_END_DATE = '2080-02-16'
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed


/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


Shape of DataFrame:  (130859, 8)


In [12]:
df.shape

(130859, 8)

In [13]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/home/ubuntu/.local/lib/python3.8/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

Successfully added technical indicators
Successfully added turbulence index


In [14]:
processed.tail()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
126107,2023-02-21,185.419998,186.360001,184.139999,184.320206,250569,TRV,1,-0.809061,193.277733,180.543288,49.359314,-56.720870,10.885409,187.348340,187.351487,21.08147
126108,2023-02-21,496.700012,501.394989,492.140015,495.309998,1014660,UNH,1,-1.465698,505.582597,470.812401,47.960983,91.820457,8.519067,487.895332,508.449635,21.08147
126109,2023-02-21,39.900002,39.980000,39.400002,39.465000,5721088,VZ,1,-0.019596,41.942736,39.260764,48.820859,-103.957206,35.771198,40.690500,39.283687,21.08147
126110,2023-02-21,36.380001,36.509998,36.090000,36.349998,1835368,WBA,1,-0.069970,37.181454,35.348825,48.743560,-11.082192,14.338233,36.135497,37.582818,21.08147
126111,2023-02-21,142.880005,148.339996,142.149994,147.273193,10743539,WMT,1,0.687985,147.640967,139.355354,55.034568,84.898881,25.315200,143.442107,145.154809,21.08147


In [15]:
processed.shape

(126112, 17)

In [16]:
len(processed.index.unique()),len(processed["date"].unique()),len(processed[processed["date"]=="2023-02-17"])

(126112, 4504, 28)

In [26]:
available_ticks=list(processed.tic.unique())
set(DOW_30_TICKER)-set(available_ticks),available_ticks

({'DOW', 'V'},
 ['AAPL',
  'AMGN',
  'AXP',
  'BA',
  'CAT',
  'CRM',
  'CSCO',
  'CVX',
  'DIS',
  'GS',
  'HD',
  'HON',
  'IBM',
  'INTC',
  'JNJ',
  'JPM',
  'KO',
  'MCD',
  'MMM',
  'MRK',
  'MSFT',
  'NKE',
  'PG',
  'TRV',
  'UNH',
  'VZ',
  'WBA',
  'WMT'])

In [130]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 28, State Space: 281


In [131]:
env_kwargs = {
    "hmax": 5, 
    "initial_amount": 500, 
    "buy_cost_pct": 0.01, 
    "sell_cost_pct": 0.01, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

In [142]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

timesteps_dict = {'a2c' : 10_0000, 
                 'ppo' : 10_00000, 
                 'ddpg' : 10_0000
                 }

In [143]:
from finrl.agents.stablebaselines3.models import *

In [144]:
class DRLAgent:
    @staticmethod
    def get_model(
        model_name,
        env,
        policy="MlpPolicy",
        policy_kwargs=None,
        model_kwargs=None,
        seed=None,
        verbose=1,
    ):
        if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")

        if model_kwargs is None:
            temp_model_kwargs = MODEL_KWARGS[model_name]
        else:
            temp_model_kwargs = model_kwargs.copy()

        if "action_noise" in temp_model_kwargs:
            n_actions = env.action_space.shape[-1]
            temp_model_kwargs["action_noise"] = NOISE[
                temp_model_kwargs["action_noise"]
            ](mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
        print(temp_model_kwargs)
        return MODELS[model_name](
            policy=policy,
            env=env,
            tensorboard_log=f"{config.TENSORBOARD_LOG_DIR}/{model_name}",
            verbose=verbose,
            policy_kwargs=policy_kwargs,
            seed=seed,
            **temp_model_kwargs,
        )

    @staticmethod
    def train_model(model, model_name, tb_log_name, iter_num, total_timesteps=5000):
        model = model.learn(
            total_timesteps=total_timesteps,
            tb_log_name=tb_log_name,
            callback=TensorboardCallback(),
        )
        model.save(
            f"{config.TRAINED_MODEL_DIR}/{model_name.upper()}_{total_timesteps // 1000}k_{iter_num}"
        )
        return model

    @staticmethod
    def get_validation_sharpe(iteration, model_name):
        """Calculate Sharpe ratio based on validation results"""
        df_total_value = pd.read_csv(
            f"results/account_value_validation_{model_name}_{iteration}.csv"
        )
        # If the agent did not make any transaction
        if df_total_value["daily_return"].var() == 0:
            if df_total_value["daily_return"].mean() > 0:
                return np.inf
            else:
                return 0.0
        else:
            return (
                (4**0.5)
                * df_total_value["daily_return"].mean()
                / df_total_value["daily_return"].std()
            )
    @staticmethod
    def get_modelWeights(model,path):
        file_list = os.listdir(path)
        sorted_list = sorted([f for f in file_list if f.lower().startswith(model)])
        if not sorted_list:
            raise("Pretrained weights not Found!!")
        else:
            print(f"Found weights {sorted_list[-1]}!!")
        return sorted_list[-1]
        

    def __init__(
        self,
        df,
        train_period,
        val_test_period,
        stock_dim,
        hmax,
        initial_amount,
        buy_cost_pct,
        sell_cost_pct,
        reward_scaling,
        state_space,
        action_space,
        tech_indicator_list,
        print_verbosity,
        use_pretrain=False,
        pretrain_pth="",
        best_model="ppo",
        reset_interval=60
        
    ):
        self.df = df
        self.train_period = train_period
        self.val_test_period = val_test_period
        self.unique_trade_date = df[(df.date > val_test_period[0]) & (df.date <= val_test_period[1])].date.unique()
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.buy_cost_pct = buy_cost_pct
        self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.print_verbosity = print_verbosity
        self.use_pretrain=use_pretrain
        self.pretrain_pth=pretrain_pth
        self.best_model=best_model
        self.reset_interval=reset_interval

    def DRL_validation(self, model, test_data, test_env, test_obs):
        """validation process"""
        for _ in range(len(test_data.index.unique())):
            action, _states = model.predict(test_obs)
            test_obs, rewards, dones, info = test_env.step(action)

    def DRL_prediction(
        self, model, name, last_state, iter_num, turbulence_threshold, initial
    ):
        """make a prediction based on trained model"""
        
        ## trading env
        trade_data = data_split(
            self.df,
            start=self.unique_trade_date[0],
            end=self.unique_trade_date[-1],
        )
        trade_env = DummyVecEnv(
            [
                lambda: StockTradingEnv(
                    df=trade_data,
                    stock_dim=self.stock_dim,
                    hmax=self.hmax,
                    initial_amount=self.initial_amount,
                    num_stock_shares=[0] * self.stock_dim,
                    buy_cost_pct=[self.buy_cost_pct] * self.stock_dim,
                    sell_cost_pct=[self.sell_cost_pct] * self.stock_dim,
                    reward_scaling=self.reward_scaling,
                    state_space=self.state_space,
                    action_space=self.action_space,
                    tech_indicator_list=self.tech_indicator_list,
                    turbulence_threshold=turbulence_threshold,
                    initial=initial,
                    previous_state=last_state,
                    model_name=name,
                    mode="trade",
                    iteration=iter_num,
                    print_verbosity=self.print_verbosity,
                )
            ]
        )

        trade_obs = trade_env.reset()

        for i in range(len(trade_data.index.unique())):
            action, _states = model.predict(trade_obs)
            #print(f"Training actions are {action}")
            trade_obs, rewards, dones, info = trade_env.step(action)
            if i == (len(trade_data.index.unique()) - 2):
                # print(env_test.render())
                last_state = trade_env.render()

        df_last_state = pd.DataFrame({"last_state": last_state})
        df_last_state.to_csv(f"results/last_state_{name}_{i}.csv", index=False)
        return last_state
    
    
    def DRL_single_prediction(
        self, model, name="ensemble", initial=True,last_state=[]
    ):
        """make a prediction based on trained model"""

        ## trading env
        trade_data = data_split(
            self.df,
            start=self.unique_trade_date[-1],
            end="2080-01-01",
        )
        trade_env = DummyVecEnv(
            [
                lambda: StockTradingEnv(
                    df=trade_data,
                    stock_dim=self.stock_dim,
                    hmax=self.hmax,
                    initial_amount=self.initial_amount,
                    num_stock_shares=[0] * self.stock_dim,
                    buy_cost_pct=[self.buy_cost_pct] * self.stock_dim,
                    sell_cost_pct=[self.sell_cost_pct] * self.stock_dim,
                    reward_scaling=self.reward_scaling,
                    state_space=self.state_space,
                    action_space=self.action_space,
                    tech_indicator_list=self.tech_indicator_list,
                    turbulence_threshold=turbulence_threshold,
                    initial=initial,
                    previous_state=last_state,
                    model_name=name,
                    mode="trade",
                    print_verbosity=self.print_verbosity,
                )
            ]
        )

        trade_obs = trade_env.reset()
        actions=[]

        for i in range(len(trade_data.index.unique())):
            action, _states = model.predict(trade_obs)
            print(f"Day Action is {action}")
            actions.append(action)

        return actions,last_state

    def run_strategy(self, A2C_model_kwargs, PPO_model_kwargs, DDPG_model_kwargs, timesteps_dict):
        i=1
        """Ensemble Strategy that combines PPO, A2C and DDPG"""
        print("============Training Best Model============")

        insample_turbulence = self.df[(self.df.date < self.train_period[1])& (self.df.date >= self.train_period[0])]
        insample_turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, 0.90)
        
        historical_turbulence = insample_turbulence.drop_duplicates(subset=["date"])
        historical_turbulence_mean = np.mean(historical_turbulence.turbulence.values)
        
        if historical_turbulence_mean > insample_turbulence_threshold:
            # if the mean of the historical data is greater than the 90% quantile of insample turbulence data
            # then we assume that the current market is volatile,
            # therefore we set the 90% quantile of insample turbulence data as the turbulence threshold
            # meaning the current turbulence can't exceed the 90% quantile of insample turbulence data
            turbulence_threshold = insample_turbulence_threshold
        else:
            # if the mean of the historical data is less than the 90% quantile of insample turbulence data
            # then we tune up the turbulence_threshold, meaning we lower the risk
            turbulence_threshold = np.quantile(
                insample_turbulence.turbulence.values, 1
            )

        #turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, 0.99)
        
        print("Turbulence_threshold: ", turbulence_threshold)

        self.turbulence_threshold=turbulence_threshold

        start = time.time()

        ############## Environment Setup starts ##############
        ## training env
        train = data_split(self.df,start=self.train_period[0],end=self.train_period[1],)
        stday = random.randint(0,len(train.index.unique())-(self.reset_interval*self.stock_dim)-2)
        self.train_env = DummyVecEnv(
            [
                lambda: StockTradingEnv2(
                    df=train,
                    stock_dim=self.stock_dim,
                    hmax=self.hmax,
                    initial_amount=self.initial_amount,
                    num_stock_shares=[0] * self.stock_dim,
                    buy_cost_pct=[self.buy_cost_pct] * self.stock_dim,
                    sell_cost_pct=[self.sell_cost_pct] * self.stock_dim,
                    reward_scaling=self.reward_scaling,
                    state_space=self.state_space,
                    action_space=self.action_space,
                    tech_indicator_list=self.tech_indicator_list,
                    print_verbosity=self.print_verbosity,
                    random_day=stday,
                    reset_interval=self.reset_interval
                )
            ]
            )
        validation = data_split(self.df,start=self.unique_trade_date[0],end=self.unique_trade_date[-1],)
        val_env = DummyVecEnv(
                [
                    lambda: StockTradingEnv(
                        df=validation,
                        stock_dim=self.stock_dim,
                        hmax=self.hmax,
                        initial_amount=self.initial_amount,
                        num_stock_shares=[0] * self.stock_dim,
                        buy_cost_pct=[self.buy_cost_pct] * self.stock_dim,
                        sell_cost_pct=[self.sell_cost_pct] * self.stock_dim,
                        reward_scaling=self.reward_scaling,
                        state_space=self.state_space,
                        action_space=self.action_space,
                        tech_indicator_list=self.tech_indicator_list,
                        turbulence_threshold=turbulence_threshold,
                        iteration=1,
                        model_name="BestModel",
                        mode="validation",
                        print_verbosity=self.print_verbosity,
                    )
                ]
            )
        val_obs = val_env.reset()
        ############## Environment Setup ends ##############
        ############## Training and Validation starts ##############
        print(
            "======Model training from: ",
            self.train_period[0],
            "to ",
            self.train_period[1]
        )
        if self.best_model=="a2c":
            model_a2c = self.get_model("a2c", self.train_env, policy="MlpPolicy", model_kwargs=A2C_model_kwargs)
            if self.use_pretrain:
                print("======Loading A2C Pretrained Model========")
                model_a2c.load(os.path.join(self.pretrain_pth,self.get_modelWeights("a2c",self.pretrain_pth)))
            print("======A2C Training========")
            model_a2c = self.train_model(
                model_a2c,
                "a2c",
                tb_log_name=f"a2c_{i}",
                iter_num=i,
                total_timesteps=timesteps_dict["a2c"],
            )  
            print("======A2C Validation from: ",
                validation_start_date,
                "to ",
                validation_end_date,)
            self.DRL_validation(
                model=model_a2c,
                test_data=validation,
                test_env=val_env,
                test_obs=val_obs,
            )
            sharpe_a2c = self.get_validation_sharpe(1, model_name="BestModel")
            print("A2C Sharpe Ratio: ", sharpe_a2c)
            self.sharpe=sharpe_a2c
            self.model=model_a2c
            
        elif self.best_model=="ppo":
            
            model_ppo = self.get_model("ppo", self.train_env, policy="MlpPolicy", model_kwargs=PPO_model_kwargs)
            if self.use_pretrain:
                print("======Loading PPO Pretrained Model========")
                model_ppo.load(os.path.join(self.pretrain_pth,self.get_modelWeights("ppo",self.pretrain_pth)))
            print("======PPO Training========")
            model_ppo = self.train_model(
                model_ppo,
                "ppo",
                tb_log_name=f"ppo_{i}",
                iter_num=i,
                total_timesteps=timesteps_dict["ppo"],
            )
            print(
                "======PPO Validation from: ",
                self.unique_trade_date[0],
                "to ",
                self.unique_trade_date[-1],
            )
            self.DRL_validation(
                model=model_ppo,
                test_data=validation,
                test_env=val_env,
                test_obs=val_obs,
            )
            sharpe_ppo = self.get_validation_sharpe(1, model_name="BestModel")
            print("PPO Sharpe Ratio: ", sharpe_ppo)
            self.sharpe=sharpe_ppo
            self.model=model_ppo
        
        elif self.best_model=="ddpg":
            
            model_ddpg = self.get_model(
                "ddpg",
                self.train_env,
                policy="MlpPolicy",
                model_kwargs=DDPG_model_kwargs,
            )
            if self.use_pretrain:
                print("======Loading DDPG Pretrained Model========")
                model_ddpg.load(os.path.join(self.pretrain_pth,self.get_modelWeights("ddpg",self.pretrain_pth)))
            print("======DDPG Training========")
            model_ddpg = self.train_model(
                model_ddpg,
                "ddpg",
                tb_log_name=f"ddpg_{i}",
                iter_num=i,
                total_timesteps=timesteps_dict["ddpg"],
            )  # 50_000
            
            self.DRL_validation(
                model=model_ddpg,
                test_data=validation,
                test_env=val_env,
                test_obs=val_obs,
            )
            sharpe_ddpg = self.get_validation_sharpe(1, model_name="BestModel")
            print("DDPg Sharpe Ratio: ", sharpe_ddpg)
            self.sharpe=sharpe_ddpg
            self.model=model_ddpg
            
        else:
            raise NotImplementedError

        ############## Trading starts ##############
        print("======Trading from: ",
            self.unique_trade_date[0],
            "to ",
            self.unique_trade_date[-1],)
        
        last_state_ensemble = self.DRL_prediction(
            model=self.model,
            name="BestModel",
            last_state=[],
            iter_num=1,
            turbulence_threshold=turbulence_threshold,
            initial=True,
        )
        
        end = time.time()
        print("Training Strategy took: ", (end - start) / 60, " minutes")
        ## Assign the model as final model

        return True

In [145]:
agent = DRLAgent(df=processed,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TEST_START_DATE,TEST_END_DATE),
                 use_pretrain=False,
                 pretrain_pth="/mnt/trained_models",
                 best_model="ppo",
                 **env_kwargs)

In [ ]:
status = agent.run_strategy(A2C_model_kwargs,
                             PPO_model_kwargs,
                             DDPG_model_kwargs,
                             timesteps_dict)

============Training Best Model============
Turbulence_threshold:  639.3259952984249
Intialize Environment StartDay: 0, ResetDay: 1680,Episode: 0
======Model training from:  2005-04-01 to  2023-02-07
{'ent_coef': 0.01, 'n_steps': 2048, 'learning_rate': 0.00025, 'batch_size': 128}
Using cuda device
======PPO Training========
Reseting Environment StartDay: 1229, ResetDay: 2909,Episode: 1
Logging to tensorboard_log/ppo/ppo_1_1
Environment reached Terminal state as number of trading days reached limit!!
Reseting Environment StartDay: 1622, ResetDay: 3302,Episode: 2
--------------------------------------
| time/              |               |
|    fps             | 78            |
|    iterations      | 1             |
|    time_elapsed    | 25            |
|    total_timesteps | 2048          |
| train/             |               |
|    reward          | 5.9591675e-05 |
--------------------------------------
Environment reached Terminal state as number of trading days reached limit!!
Rese

---------------------------------------
| time/                   |           |
|    fps                  | 75        |
|    iterations           | 9         |
|    time_elapsed         | 245       |
|    total_timesteps      | 18432     |
| train/                  |           |
|    approx_kl            | 41.663143 |
|    clip_fraction        | 0.717     |
|    clip_range           | 0.2       |
|    entropy_loss         | -41       |
|    explained_variance   | -4.31     |
|    learning_rate        | 0.00025   |
|    loss                 | -0.256    |
|    n_updates            | 80        |
|    policy_gradient_loss | 0.141     |
|    reward               | 0.0       |
|    std                  | 1.05      |
|    value_loss           | 0.00399   |
---------------------------------------
Environment reached Terminal state as number of trading days reached limit!!
Reseting Environment StartDay: 1629, ResetDay: 3309,Episode: 12
Environment reached Terminal state as number of trading day

Environment reached Terminal state as number of trading days reached limit!!
day: 4310, episode: 20
begin_total_asset: 500.00
end_total_asset: 31.37
total_reward: -468.63
total_cost: 384.10
total_trades: 27612
Sharpe: -2.043
Reseting Environment StartDay: 2615, ResetDay: 4295,Episode: 21
--------------------------------------------
| time/                   |                |
|    fps                  | 75             |
|    iterations           | 17             |
|    time_elapsed         | 464            |
|    total_timesteps      | 34816          |
| train/                  |                |
|    approx_kl            | 123.51064      |
|    clip_fraction        | 0.751          |
|    clip_range           | 0.2            |
|    entropy_loss         | -41.9          |
|    explained_variance   | -10.2          |
|    learning_rate        | 0.00025        |
|    loss                 | -0.291         |
|    n_updates            | 160            |
|    policy_gradient_loss | 0.146   

Environment reached Terminal state as number of trading days reached limit!!
day: 3515, episode: 30
begin_total_asset: 500.00
end_total_asset: 57.33
total_reward: -442.67
total_cost: 518.36
total_trades: 35105
Sharpe: -1.572
Reseting Environment StartDay: 1681, ResetDay: 3361,Episode: 31
--------------------------------------------
| time/                   |                |
|    fps                  | 74             |
|    iterations           | 25             |
|    time_elapsed         | 683            |
|    total_timesteps      | 51200          |
| train/                  |                |
|    approx_kl            | 41.57265       |
|    clip_fraction        | 0.727          |
|    clip_range           | 0.2            |
|    entropy_loss         | -43            |
|    explained_variance   | -7.47          |
|    learning_rate        | 0.00025        |
|    loss                 | -0.296         |
|    n_updates            | 240            |
|    policy_gradient_loss | 0.127   

Environment reached Terminal state as number of trading days reached limit!!
Reseting Environment StartDay: 2719, ResetDay: 4399,Episode: 40
Environment reached Terminal state as number of trading days reached limit!!
day: 4399, episode: 40
begin_total_asset: 500.00
end_total_asset: 336.05
total_reward: -163.95
total_cost: 242.19
total_trades: 41699
Sharpe: -0.006
Reseting Environment StartDay: 2810, ResetDay: 4490,Episode: 41
-------------------------------------------
| time/                   |               |
|    fps                  | 74            |
|    iterations           | 33            |
|    time_elapsed         | 902           |
|    total_timesteps      | 67584         |
| train/                  |               |
|    approx_kl            | 110.878876    |
|    clip_fraction        | 0.733         |
|    clip_range           | 0.2           |
|    entropy_loss         | -44.2         |
|    explained_variance   | -11.7         |
|    learning_rate        | 0.00025      

...

Environment reached Terminal state as number of trading days reached limit!!
Reseting Environment StartDay: 253, ResetDay: 1933,Episode: 187
--------------------------------------------
| time/                   |                |
|    fps                  | 74             |
|    iterations           | 153            |
|    time_elapsed         | 4203           |
|    total_timesteps      | 313344         |
| train/                  |                |
|    approx_kl            | 91.07977       |
|    clip_fraction        | 0.758          |
|    clip_range           | 0.2            |
|    entropy_loss         | -59.4          |
|    explained_variance   | 0.0622         |
|    learning_rate        | 0.00025        |
|    loss                 | -0.438         |
|    n_updates            | 1520           |
|    policy_gradient_loss | 0.762          |
|    reward               | -7.0860675e-05 |
|    std                  | 2.03           |
|    value_loss           | 2.03e-06       |
----

Environment reached Terminal state as number of trading days reached limit!!
day: 4072, episode: 195
begin_total_asset: 500.00
end_total_asset: 958.13
total_reward: 458.13
total_cost: 4.93
total_trades: 47031
Sharpe: 0.606
Reseting Environment StartDay: 182, ResetDay: 1862,Episode: 196
Environment reached Terminal state as number of trading days reached limit!!
Reseting Environment StartDay: 2638, ResetDay: 4318,Episode: 197
-------------------------------------------
| time/                   |               |
|    fps                  | 74            |
|    iterations           | 161           |
|    time_elapsed         | 4423          |
|    total_timesteps      | 329728        |
| train/                  |               |
|    approx_kl            | 50.10978      |
|    clip_fraction        | 0.732         |
|    clip_range           | 0.2           |
|    entropy_loss         | -60.5         |
|    explained_variance   | 0.204         |
|    learning_rate        | 0.00025       |

Environment reached Terminal state as number of trading days reached limit!!
day: 2604, episode: 205
begin_total_asset: 500.00
end_total_asset: 486.73
total_reward: -13.27
total_cost: 4.95
total_trades: 47031
Sharpe: 0.248
Reseting Environment StartDay: 2171, ResetDay: 3851,Episode: 206
--------------------------------------------
| time/                   |                |
|    fps                  | 74             |
|    iterations           | 169            |
|    time_elapsed         | 4644           |
|    total_timesteps      | 346112         |
| train/                  |                |
|    approx_kl            | 70.555405      |
|    clip_fraction        | 0.691          |
|    clip_range           | 0.2            |
|    entropy_loss         | -61.6          |
|    explained_variance   | -0.187         |
|    learning_rate        | 0.00025        |
|    loss                 | -0.426         |
|    n_updates            | 1680           |
|    policy_gradient_loss | 0.232    

Environment reached Terminal state as number of trading days reached limit!!
day: 4293, episode: 215
begin_total_asset: 500.00
end_total_asset: 888.37
total_reward: 388.37
total_cost: 4.92
total_trades: 47031
Sharpe: 0.509
Reseting Environment StartDay: 1447, ResetDay: 3127,Episode: 216
--------------------------------------------
| time/                   |                |
|    fps                  | 74             |
|    iterations           | 177            |
|    time_elapsed         | 4865           |
|    total_timesteps      | 362496         |
| train/                  |                |
|    approx_kl            | 34.232132      |
|    clip_fraction        | 0.704          |
|    clip_range           | 0.2            |
|    entropy_loss         | -62.7          |
|    explained_variance   | 0.156          |
|    learning_rate        | 0.00025        |
|    loss                 | -0.471         |
|    n_updates            | 1760           |
|    policy_gradient_loss | 0.182    

Environment reached Terminal state as number of trading days reached limit!!
day: 1686, episode: 225
begin_total_asset: 500.00
end_total_asset: 362.07
total_reward: -137.93
total_cost: 5.37
total_trades: 47037
Sharpe: 0.026
Reseting Environment StartDay: 2173, ResetDay: 3853,Episode: 226
-------------------------------------------
| time/                   |               |
|    fps                  | 74            |
|    iterations           | 185           |
|    time_elapsed         | 5085          |
|    total_timesteps      | 378880        |
| train/                  |               |
|    approx_kl            | 170.30782     |
|    clip_fraction        | 0.733         |
|    clip_range           | 0.2           |
|    entropy_loss         | -63.9         |
|    explained_variance   | -0.316        |
|    learning_rate        | 0.00025       |
|    loss                 | -0.583        |
|    n_updates            | 1840          |
|    policy_gradient_loss | 0.185         |
|    re